In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
import random
import math
from sklearn.neural_network import MLPClassifier

In [2]:
# df_patients = pd.read_csv('Data Alpha Bravo Charlie Delta Echo - raw cleaned.csv', sep=';')  # cleaned data
# df_patients = pd.read_csv('Data Alpha Bravo Charlie Delta Echo - supercleaned.csv', sep=';')  # super cleaned data
df_patients = pd.read_csv('Data Alpha Bravo Charlie Delta Echo - Super cleaned testset.csv', sep=';')  # super cleaned testset
df_patients['Bias'] = 0

In [3]:
def SplitData(df, xcol, ycol, percentage):
    allIndex = np.unique(df['pat'].tolist()) # Get all unique patients id's
    random.seed(2) # Set random seeed so the answer is the same
    
    PercentageIndex = []
    for i in np.unique(np.floor(allIndex / 1000)): # Cycle through each categorie (only cycles through the ones that are present)
        CatPatients = allIndex[allIndex<((i+1)*1000)] # Filters out patients who are bigger then our max
        CatPatients = CatPatients[CatPatients>=((i)*1000)] # Filters out patients that are smaller
        
        AmountItems = len(CatPatients)
        AmountRandom = math.floor(AmountItems*percentage)
        PercentageIndex.extend(random.sample(list(CatPatients), AmountRandom))        
        
    AmountItems = len(allIndex)
    AmountRandom = math.floor(AmountItems*percentage)
    
    PercentageIndex = random.sample(list(allIndex), AmountRandom)
    
    # Normalize data
    xcoldf = df[xcol]
#     xcoldf = normalize(xcoldf)
    
    Percentagedf = xcoldf[df['pat'].isin(PercentageIndex)]
    Percentagey = df[ycol][df['pat'].isin(PercentageIndex)]
    
    Testdf = xcoldf[~df['pat'].isin(PercentageIndex)]
    Testy = df[ycol][~df['pat'].isin(PercentageIndex)]
    
    testpatlist = df['pat'][~df['pat'].isin(PercentageIndex)]
    return (Percentagedf, Testdf, Percentagey, Testy, testpatlist)

In [4]:
# Get all columns except three as X
columns = list(df_patients.columns)
# columns.remove('Unnamed: 0')
columns.remove('cat')
columns.remove('pat')
param = columns

Xcolumns = []
Xcolumns.extend(param)

X = df_patients[Xcolumns]
y = df_patients[['cat']]

# Split training set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

X_train, X_test, y_train, y_test, tmptestpatlist = SplitData(df_patients, Xcolumns, 'cat', 0.8)

In [5]:
# Learn model
lr = LogisticRegression(random_state=21)
# lr = MLPClassifier(random_state=21,tol=0.00001,learning_rate='adaptive',verbose=10)
lr.fit(X_train, y_train)

#Predict test set
y_pred = lr.predict(X_test)

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [6]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test, y_pred, digits=3))
print(accuracy_score(y_test, y_pred, normalize=False, sample_weight=None))

              precision    recall  f1-score   support

           1      0.000     0.000     0.000        32
           2      0.816     0.691     0.748       243
           3      0.412     0.219     0.286        32
           4      0.000     0.000     0.000         0

   micro avg      0.570     0.570     0.570       307
   macro avg      0.307     0.228     0.259       307
weighted avg      0.688     0.570     0.622       307

175


/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [7]:
tmpdf = pd.DataFrame()
tmpdf['pat'] = tmptestpatlist
tmpdf['pred'] = y_pred
tmpdf.groupby('pat')['pred'].mean()

pat
1025    2.000000
2013    1.773663
3036    3.406250
Name: pred, dtype: float64

In [8]:
for num, i in enumerate(lr.coef_):
    print(num)
    ParamCheck = pd.DataFrame()
    ParamCheck['Params'] = Xcolumns
    ParamCheck['Theta'] = list(i)
    ParamCheck['Absolute'] = np.absolute(list(ParamCheck['Theta']))
    ParamCheck = ParamCheck.sort_values(by=['Absolute'], ascending=False)
    print(ParamCheck.head())

0
             Params     Theta  Absolute
388  Delta_EnergyZR -0.006546  0.006546
2    Alpha_EnergyXR  0.006499  0.006499
3    Alpha_EnergyYR -0.006358  0.006358
134  Bravo_EnergyYL -0.005600  0.005600
6    Alpha_EnergyYL -0.005444  0.005444
1
               Params     Theta  Absolute
517     Echo_EnergyXL  0.005290  0.005290
135    Bravo_EnergyZL -0.004067  0.004067
258  Charlie_EnergyXR -0.003348  0.003348
263  Charlie_EnergyZL  0.003145  0.003145
388    Delta_EnergyZR -0.002885  0.002885
2
             Params     Theta  Absolute
135  Bravo_EnergyZL -0.006161  0.006161
388  Delta_EnergyZR  0.004834  0.004834
390  Delta_EnergyYL -0.004040  0.004040
517   Echo_EnergyXL -0.003825  0.003825
131  Bravo_EnergyYR  0.003069  0.003069
3
               Params     Theta  Absolute
7      Alpha_EnergyZL -0.005693  0.005693
258  Charlie_EnergyXR -0.005221  0.005221
517     Echo_EnergyXL -0.004908  0.004908
134    Bravo_EnergyYL  0.004238  0.004238
2      Alpha_EnergyXR -0.003578  0.003578


In [9]:
print('hi')

hi
